## Packages

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta


### CSV File Import

In [2]:
### Data
name = 'Workout Progress Data - master.csv'
workout_data = pd.read_csv(name)

name = 'Exercise_list.csv'
exercise_file = pd.read_csv(name)


## Fields

In [5]:
# Date Parameters 

# Calculations
today = date.today()
startOfWeek = today - relativedelta(days=today.weekday())
endOfWeek = startOfWeek + relativedelta(days=6)
firstDayOfMonth = today.replace(day=1)
nextMonthDate = today + relativedelta(months=1, day=1)

# Formatting into string
today = today.strftime('%m-%d-%Y')

startOfWeek = startOfWeek.strftime('%m-%d-%Y')
end_of_week = endOfWeek.strftime('%m-%d-%Y')

firstDayOfMonth = firstDayOfMonth.strftime('%m-%d-%Y')
nextMonthDate = nextMonthDate.strftime('%m-%d-%Y')

print("Today: " + str(today) + '\n')
print("Week Start: " + str(startOfWeek))
print("Week End: " + str(end_of_week) + '\n')
print("Month Start: " + str(firstDayOfMonth))
print("Next Month Start: " + str(nextMonthDate))


Today: 11-27-2022

Week Start: 11-21-2022
Week End: 11-27-2022

Month Start: 11-01-2022
Next Month Start: 12-01-2022


In [6]:
workout_data = workout_data.sort_values(by=['Date','Exercise','Sets'])
workout_data.head()

,Date,Exercise,Muscle Group,Sets,Reps,Weight,Duration,Note
56,11/23/22,Bicep Curl,NaN,1.0,10.0,15,NaN,NaN
57,11/23/22,Bicep Curl,NaN,2.0,10.0,20,NaN,NaN
58,11/23/22,Bicep Curl,NaN,3.0,10.0,20,NaN,NaN
59,11/23/22,Bicep Curl,NaN,4.0,10.0,15,NaN,NaN
60,11/23/22,Bicep Curl,NaN,5.0,10.0,20,NaN,NaN


In [8]:
# Unique days count
uniqueDaysList = pd.DataFrame(workout_data['Date'].unique(), columns=['Date'])
total_count_unique_days = len(uniqueDaysList)
print(f"Count of unique days in all csv: {total_count_unique_days}")


Count of unique days in all csv: 5


---

## Current Month

In [9]:
currentMonthData = workout_data[workout_data['Date'] >= firstDayOfMonth]
currentMonthData


,Date,Exercise,Muscle Group,Sets,Reps,Weight,Duration,Note
56,11/23/22,Bicep Curl,NaN,1.0,10.0,15,NaN,NaN
57,11/23/22,Bicep Curl,NaN,2.0,10.0,20,NaN,NaN
58,11/23/22,Bicep Curl,NaN,3.0,10.0,20,NaN,NaN
59,11/23/22,Bicep Curl,NaN,4.0,10.0,15,NaN,NaN
60,11/23/22,Bicep Curl,NaN,5.0,10.0,20,NaN,NaN
...,...,...,...,...,...,...,...,...
161,11/27/22,Lat Raise,NaN,2.0,10.0,15,NaN,NaN
162,11/27/22,Lat Raise,NaN,3.0,10.0,20,NaN,NaN
163,11/27/22,Lat Raise,NaN,4.0,5.0,20,NaN,NaN
143,11/27/22,Pull Up,NaN,1.0,8.0,BW,NaN,NaN


In [10]:
# For loop over list of unique dates in curentMonthData dates's column
# Counting each loop as a Record Day
daysCount = 0
for date in currentMonthData[currentMonthData['Date'] >= firstDayOfMonth]['Date'].unique():
    print(date)
    daysCount += 1

11/23/22
11/24/22
11/25/22
11/26/22
11/27/22


In [11]:
print(f"Been to the gym -> {daysCount} days this month.")

Been to the gym -> 5 days this month.


In [12]:
lastRecordDate = currentMonthData['Date'].unique()[-1]
lastRecordData = currentMonthData[currentMonthData['Date'] == lastRecordDate]

lastRecordData

,Date,Exercise,Muscle Group,Sets,Reps,Weight,Duration,Note
142,11/27/22,Bodyweight,NaN,NaN,NaN,127.1,NaN,NaN
145,11/27/22,Chin Up,NaN,1.0,5.0,BW,NaN,NaN
146,11/27/22,Dips,NaN,1.0,10.0,BW,NaN,NaN
147,11/27/22,Dips,NaN,2.0,10.0,BW,NaN,NaN
148,11/27/22,Dips,NaN,3.0,10.0,BW,NaN,NaN
149,11/27/22,Dumbbell Shoulder Press,NaN,1.0,10.0,35,NaN,NaN
150,11/27/22,Dumbbell Shoulder Press,NaN,2.0,10.0,35,NaN,NaN
151,11/27/22,Dumbbell Shoulder Press,NaN,3.0,10.0,45,NaN,NaN
152,11/27/22,Dumbbell Shoulder Press,NaN,4.0,8.0,45,NaN,NaN
153,11/27/22,Dumbbell Shoulder Press,NaN,5.0,10.0,35,NaN,NaN


In [234]:
# what muscle did we work out last
for unique_muscle in lastRecordData['Muscle Group'].unique():
    print(unique_muscle)

Back
Shoulders
Quadriceps


In [237]:
lastRecordDate2 = currentMonthData['Date'].unique()[-2]
lastRecordData2 = currentMonthData[currentMonthData['Date'] == lastRecordDate2]
# what muscle did we work out previous to last
for unique_muscle in lastRecordData2['Muscle Group'].unique():
    print(unique_muscle)

Body Weight
